In [5]:
## Import Packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
import seaborn as sns
sns.set()

In [8]:
## Scrape data for a specific powerlifting meet for lifter statistics
def return_df(x):
    division= []
    place = []
    lifter_name = []
    sex= []
    age = []
    equip = []
    weight_class = []
    weight=[]
    squat = []
    bench = []
    deadlift = []
    total = []
    dots = []
    data = x.find_all('td')
    for i in range(len(x.find_all('td'))):
        if str(data[i]).find('divheader') > -1:
            division.append(data[i].text)
            cur_div = data[i].text
        if str(data[i-1]).find('divheader') > -1 or (str(data[i-3]).find('td class="deadlift"') > -1 and str(data[i]).find('divheader')==-1):
            place.append(data[i].text)
        if str(data[i]).find('href') > -1:
            lifter_name.append(data[i].find('a',href=True).text)
            if str(data[i-2]).find('divheader') == -1:
                division.append(cur_div)
        if data[i].text == 'F' or data[i].text == 'M':
            sex.append(data[i].text)
        if data[i-1].text =='F' or data[i-1].text =='M':
            age.append(data[i].text)
        if data[i-2].text =='F' or data[i-2].text =='M':
            equip.append(data[i].text)
        if data[i-3].text =='F' or data[i-3].text =='M':
            weight_class.append(data[i].text)
        if data[i-4].text =='F' or data[i-4].text =='M':
            weight.append(data[i].text)
        if str(data[i]).find('td class="squat"') > -1:
            if data[i].text == '':
                squat.append(0)
            else:
                squat.append(data[i].text)
        if str(data[i]).find('td class="bench"') > -1:
            if data[i].text =='':
                bench.append(0)
            else:
                bench.append(data[i].text)
        if str(data[i]).find('td class="deadlift"') > -1:
            if data[i].text == '':
                deadlift.append(0)
            else:
                deadlift.append(data[i].text)
        if data[i-8].text =='F' or data[i-8].text =='M':
            total.append(data[i].text)
        if data[i-9].text =='F' or data[i-9].text =='M':
            dots.append(data[i].text)
    df = pd.DataFrame([division,lifter_name,place,sex,age,equip,weight_class,weight,squat,bench,deadlift,total,dots],index=['Division','Lifter Name','Place','Sex', 'Age', 'Equip','Weight Class','Weight (KG)',
                                                                                                                          'Squat (KG)','Bench (KG)','Deadlift (KG)','Total (KG)','Dots']).T
    try:
        attr = x.find_all('p')[0].text.split(',')
        df['Country'] = attr[1].strip(' ').split('-')[0]
        if len(attr[1].strip(' ').split('-')) > 1:
            df['State'] = attr[1].strip(' ').split('-')[1]
        elif len(attr[1].strip(' ').split('-')) ==1:
            df['State'] = None
        df['Date'] = attr[0]
        df['YM'] = pd.to_datetime(attr[0]).strftime('%Y/%m')
        if len(attr) >2:
            df['City'] = attr[2]
        elif len(attr) <3:
            df['City'] = None
    except:
        df['Country'] = None
        df['Date'] = None
        df['YM'] = None

    
    df['Number of Lifters'] = len(squat)
    return df

def return_data(years, url):
    dataframes=[]
    # Loops through all years
    for year in years:
        # Grabs all meets for that year
        url_year = url + str(year)
        page = requests.get(url_year)
        soup = BeautifulSoup(page.text)
        all_meets = soup.find_all('a',href=True)[13:]
        # Scrapes all fields needed and appends it to a dataframe
        for meet in all_meets:
            url_meet = 'https://www.openpowerlifting.org/' + meet['href'] + '/by-division'
            page_meet = requests.get(url_meet)
            soup_meet = BeautifulSoup(page_meet.text)
            DF = return_df(soup_meet)
            DF['Competition'] = meet.text
            dataframes.append(DF)
    return pd.concat(dataframes).reset_index().drop('index',axis=1)
min_uspa = 2010
min_usapl = 1997
cur_year = datetime.datetime.now().year
years_uspa = np.arange(min_uspa, cur_year, 1)
years_usapl = np.arange(min_usapl, cur_year, 1)

df_uspa = return_data(years_uspa,'https://www.openpowerlifting.org/mlist/uspa/')
df_usapl = return_data(years_usapl,'https://www.openpowerlifting.org/mlist/usapl/')

In [48]:
df_uspa.head()

,Division,Lifter Name,Place,Sex,Age,Equip,Weight Class,Weight (KG),Squat (KG),Bench (KG),Deadlift (KG),Total (KG),Dots,Country,State,Date,YM,City,Number of Lifters,Competition
0,Women Raw 75 Open,Liz Pagel,1,F,28,Raw,75,73.7,97.5,72.5,137.5,307.5,295.56,USA,CA,2010-12-11,2010/12,Camp Pendleton\n \nDownload as CSV\n\n,113,Camp Pendleton Powerlifting Championship
1,Women Raw 90+ Open,I-Esha Scott,1,F,27,Raw,90+,91.4,142.5,85.0,155.0,382.5,328.44,USA,CA,2010-12-11,2010/12,Camp Pendleton\n \nDownload as CSV\n\n,113,Camp Pendleton Powerlifting Championship
2,Men Raw 75 Juniors 13-15,Zack Smith,1,M,15,Raw,75,69.7,82.5,60.0,117.5,260.0,195.35,USA,CA,2010-12-11,2010/12,Camp Pendleton\n \nDownload as CSV\n\n,113,Camp Pendleton Powerlifting Championship
3,Men Raw 75 Juniors 16-17,John Hansen Jr,DQ,M,17,Raw,75,74.8,142.5,NaN,172.5,0.0,,USA,CA,2010-12-11,2010/12,Camp Pendleton\n \nDownload as CSV\n\n,113,Camp Pendleton Powerlifting Championship
4,Men Raw 125 Juniors 16-17,Tyler Sage,1,M,17,Raw,125,122.8,227.5,152.5,217.5,597.5,341.73,USA,CA,2010-12-11,2010/12,Camp Pendleton\n \nDownload as CSV\n\n,113,Camp Pendleton Powerlifting Championship


In [56]:
df_uspa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169594 entries, 0 to 169593
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Division           169594 non-null  object        
 1   Lifter Name        169594 non-null  object        
 2   Place              169594 non-null  object        
 3   Sex                169593 non-null  object        
 4   Age                169593 non-null  object        
 5   Equip              169593 non-null  object        
 6   Weight Class       169593 non-null  object        
 7   Weight (KG)        169593 non-null  object        
 8   Squat (KG)         129253 non-null  float64       
 9   Bench (KG)         150338 non-null  float64       
 10  Deadlift (KG)      146806 non-null  float64       
 11  Total (KG)         169593 non-null  float64       
 12  Dots               169593 non-null  object        
 13  Country            169594 non-null  object  

In [62]:
df_uspa.isnull().sum()


Division                 0
Lifter Name              0
Place                    0
Sex                      1
Age                      1
Equip                    1
Weight Class             1
Weight (KG)              1
Squat (KG)           40341
Bench (KG)           19256
Deadlift (KG)        22788
Total (KG)               1
Dots                     1
Country                  0
State                    0
Date                     0
YM                       0
City                    99
Number of Lifters        0
Competition              0
dtype: int64

In [64]:
df_uspa.Age.unique()

array(['28', '27', '15', '17', '20', '21', '22', '23', '44', '45', '52',
       '53', '29', '31', '24', '25', '32', '26', '35', '36', '12', '40',
       '46', '65', '30', '38', '64', '59', '33', '39', '19', '42', '62',
       '60', '49', '47', '50', '54', '63', '41', '70', '51', '34', '48',
       '37', '18', '58', '55', '56', '57', '61', '16', '43', '', '18~',
       '53~', '27~', '24~', '20~', '16~', '69', '14', '13', '71', '74',
       '66', '68', '21~', '22~', '41~', '52~', '31~', '29~', '43~', '46~',
       '40~', '45~', '50~', '67~', '26~', '32~', '47~', '73', '76', '67',
       '75~', '77', '72', '80', '75', '79', '56~', '57~', '33~', '36~',
       '81', '82', '11', '30~', '78', '9', '60~', '10', '44~', '83', None,
       '90', '6', '84', '89', '85', '8', '87', '7', '86', '5', '94',
       '37~', '62~', '25~', '95', '42~'], dtype=object)

## Basic EDA Analysis
- We see that that the column "Division" contains information that can be separated out into multiple different features, such as the Age Division (Juniors, Open, Masters, etc) and Division Category (Full-power, bench-only, etc). 
- We see that we have to correct data types for certain columns such as Age, Dots, Weight & Total. 
- There are many nulls in the Squat, Bench & Deadlift columns which corresponds with lifters who eitehr failed that attempt, or did not perform that lift because they competed in a specific event such as a "Bench-Only" event or a "Push-Pull" Event.

In [50]:
## Function to extract Age Division feature from the "Division" Column for the USAPL Federation
def return_age_division_usapl(x):
    hypen = x.find('-')
    plus = x.find('+')
    hs = x.find('HS')
    if x.lower().find('sub') > -1 :
        if x.lower().find('push') > -1:
            return "Submasters"
        elif hypen ==-1:
            return "Submasters"
        return "Submasters " + str(x[hypen-2:hypen+3])
    elif x.lower().find('masters') > -1:
        if x.lower().find('99.7') > -1:
            return "Masters"
        if plus > -1 and x.lower().find('push') > -1:
            return ' '.join(str("Masters " + str(x[plus-3:plus+1])).split())
        elif hypen > -1:
            return "Masters " + str(x[hypen-2:hypen+3])
        else:
            return ' '.join(str("Masters " + str(x[plus-3:plus+1])).split())
    elif x.lower().find('junior') > -1:
        if x.lower().find('hs') > -1:
            return x[hs:]
        elif x.lower().find('push')>-1:
            return 'Junior'
        elif hypen== -1:
            return "Junior"
        return "Junior " + str(x[hypen-2:hypen+3])
    elif x.lower().find('youth') > -1:
        return 'Youth'
    else: 
        return 'Open'
    
## Function to extract Age Division feature from the "Division" Column for the USPA Federation
def return_age_division_uspa(x):
    hypen = x.find('-')
    plus = x.find('+')
    if x.lower().find('sub') > -1 :
        return "Submasters " + str(x[hypen-2:hypen+3])
    elif x.lower().find('masters') > -1:
        if x.lower().find('99.7') > -1:
            return "Masters"
        if plus > -1 and x.lower().find('push') > -1:
            return ' '.join(str("Masters " + str(x[plus-3:plus+1])).split())
        elif hypen > -1:
            return "Masters " + str(x[hypen-2:hypen+3])
        else:
            return ' '.join(str("Masters " + str(x[plus-3:plus+1])).split())
    elif x.lower().find('junior') > -1:
        return "Junior " + str(x[hypen-2:hypen+3])
    elif x.lower().find('youth') > -1:
        return 'Youth'
    else: 
        return 'Open'

## Function to Extract Division Category feature from the "Division" Column
def return_category(x):
    if x.lower().find('bench only') > -1:
        return 'Bench Only'
    elif x.lower().find('deadlift only') > -1:
        return 'Deadlift Only'
    elif x.lower().find('push') > -1:
        return ('Push-Pull')
    else:
        return('Full-Power')
    
## Function to classify lifters performance based on their totals and the federations lifting standards
def classification (sex, event, wc, equip, ac,total):
    if sex=='M':
        if event =='Full-Power' and wc =='52' and equip=='Raw' and ac=='Open':
            if total > 529.2 and total <=606.3:
                return 'Class IV'
            elif total >606.3 and total <= 700:
                return 'Class III'
            elif total > 700 and total <= 793.7:
                return 'Class II'
            elif total >793.7 and total <= 881.9:
                return 'Class 1'
            elif  total > 881.9 and total <= 959.1:
                return 'Master'
            elif  total > 959.1 and total <= 1041.7:
                return 'Elite' 
            elif total > 1041.7:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='56' and equip=='Raw' and ac=='Open':
            if total > 573.3 and total <=661.4:
                return 'Class IV'
            elif total >661.4 and total <= 755.1:
                return 'Class III'
            elif total > 755.1 and total <= 859.9:
                return 'Class II'
            elif total >859.9 and total <= 959.1:
                return 'Class 1'
            elif total > 959.1 and total <= 1041.7:
                return 'Master'
            elif total > 1041.7 and total <= 1135.4:
                return 'Elite' 
            elif total > 1135.4:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='60' and equip=='Raw' and ac=='Open':
            if total > 617.3 and total <= 711:
                return 'Class IV'
            elif total > 711 and total <= 843.3:
                return 'Class III'
            elif total > 843.3 and total <= 920.5:
                return 'Class II'
            elif total > 920.5 and total <= 1030.7:
                return 'Class 1'
            elif total > 1030.7 and total <= 1118.9:
                return 'Master'
            elif total > 118.9 and total <= 1223.6:
                return 'Elite' 
            elif  total > 1223.6:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='67.5' and equip=='Raw' and ac=='Open':
            if total > 694.5 and total <= 799.2:
                return 'Class IV'
            elif total > 799.2 and total <= 909.5:
                return 'Class III'
            elif total > 909.5 and total <= 1036.2:
                return 'Class II'
            elif total > 1036.2 and total <= 1152:
                return 'Class 1'
            elif total > 1152 and total <= 1256.7:
                return 'Master'
            elif total > 1256.7 and total <= 1366.9:
                return 'Elite' 
            elif total > 1366.9:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='75' and equip=='Raw' and ac=='Open':
            if total > 755.1 and total <= 870.9:
                return 'Class IV'
            elif total > 870.9 and total <= 992.1:
                return 'Class III'
            elif total > 992.1 and total <= 1129.9:
                return 'Class II'
            elif total > 1129.9 and total <= 1262.2:
                return 'Class 1'
            elif total > 1262.2 and total <= 1372.4:
                return 'Master'
            elif total > 1372.4 and total <= 1499.2:
                return 'Elite' 
            elif  total > 1499.2:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='82.5' and equip=='Raw' and ac=='Open':
            if total > 815.8 and total <= 931.5:
                return 'Class IV'
            elif total > 931.5 and total <= 1069.3:
                return 'Class III'
            elif total > 1069.3 and total <= 1212.6:
                return 'Class II'
            elif total > 1212.6 and total <= 1355.9:
                return 'Class 1'
            elif total > 1355.9 and total <= 1477.1:
                return 'Master'
            elif total > 1477.1 and total <= 1609.4:
                return 'Elite' 
            elif total > 1609.4:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='90' and equip=='Raw' and ac=='Open':
            if total > 859.9 and total <= 986.6:
                return 'Class IV'
            elif total > 986.6 and total <= 1129.9:
                return 'Class III'
            elif total > 1129.9 and total <= 1278.7:
                return 'Class II'
            elif total > 1278.7 and total <= 1438.6:
                return 'Class 1'
            elif total > 1438.6 and total <= 1559.8:
                return 'Master'
            elif total > 1559.8 and total <= 1697.6:
                return 'Elite' 
            elif total > 1697.6:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='100' and equip=='Raw' and ac=='Open':
            if total > 903.9 and total <= 1041.7:
                return 'Class IV'
            elif total > 1041.7 and total <= 1190.5:
                return 'Class III'
            elif total > 1190.5 and total <= 1355.9:
                return 'Class II'
            elif total > 1355.9 and total <= 1510.2:
                return 'Class 1'
            elif total > 1510.2 and total <= 1642.5:
                return 'Master'
            elif total > 1642.5 and total <= 1791.3:
                return 'Elite' 
            elif total > 1791.3:
                return 'Int Elite'

        if event =='Full-Power' and wc =='110' and equip=='Raw' and ac=='Open':
            if total > 942.5 and total <= 1074.8:
                return 'Class IV'
            elif total > 1074.8 and total <= 1229.1:
                return 'Class III'
            elif total > 1229.1 and total <= 1400:
                return 'Class II'
            elif total > 1400 and total <= 1559.8:
                return 'Class 1'
            elif total > 1559.8 and total <= 1703.1:
                return 'Master'
            elif total > 1703.1 and total <= 1851.9:
                return 'Elite' 
            elif total > 1851.9:
                return 'Int Elite'

        if event =='Full-Power' and wc =='125' and equip=='Raw' and ac=='Open':
            if total > 970.1 and total <= 1107.9:
                return 'Class IV'
            elif total > 1107.9 and total <= 1267.7:
                return 'Class III'
            elif total > 1267.7 and total <= 1438.6:
                return 'Class II'
            elif total > 1438.6 and total <= 1609.4:
                return 'Class 1'
            elif total > 1609.4 and total <= 1752.7:
                return 'Master'
            elif total > 1752.7 and total <= 1907:
                return 'Elite' 
            elif total > 1907:
                return 'Int Elite'

        if event =='Full-Power' and wc =='140' and equip=='Raw' and ac=='Open':
            if total > 1008.7 and total <= 1152:
                return 'Class IV'
            elif total > 1152 and total <= 1322.8:
                return 'Class III'
            elif total > 1322.8 and total <= 1504.7:
                return 'Class II'
            elif total > 1504.7 and total <= 1670.1:
                return 'Class 1'
            elif total > 1670.1 and total <= 1829.9:
                return 'Master'
            elif total > 1829.9 and total <= 1995.2:
                return 'Elite' 
            elif total > 1995.2:
                return 'Int Elite'

        if event =='Full-Power' and wc =='140+' and equip=='Raw' and ac=='Open':
            if total > 1036.2 and total <= 1185:
                return 'Class IV'
            elif total > 1185 and total <= 1366.9:
                return 'Class III'
            elif total > 1366.9 and total <= 1548.8:
                return 'Class II'
            elif total > 1548.8 and total <= 1719.7:
                return 'Class 1'
            elif total > 1719.7 and total <= 1885:
                return 'Master'
            elif total > 1885 and total <= 2055.9:
                return 'Elite' 
            elif total > 2055.9:
                return 'Int Elite'
    if sex=='F':
        if event =='Full-Power' and wc =='52' and equip=='Raw' and ac=='Open':
            if total > 363.8 and total <= 418.9:
                return 'Class IV'
            elif total > 418.9 and total <= 479.6:
                return 'Class III'
            elif total > 479.6 and total <= 540.2:
                return 'Class II'
            elif total > 540.2 and total <= 600.8:
                return 'Class 1'
            elif  total > 600.8 and total <= 661.4:
                return 'Master'
            elif  total > 661.4 and total <= 722.1:
                return 'Elite' 
            elif total > 722.1:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='56' and equip=='Raw' and ac=='Open':
            if total > 380.3 and total <=446.5:
                return 'Class IV'
            elif total > 446.5 and total <= 512.6:
                return 'Class III'
            elif total > 512.6 and total <= 573.3:
                return 'Class II'
            elif total > 573.3 and total <= 639.4:
                return 'Class 1'
            elif total > 639.4 and total <= 705.5:
                return 'Master'
            elif total > 705.5 and total <= 766.2:
                return 'Elite' 
            elif total > 766.2:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='60' and equip=='Raw' and ac=='Open':
            if total > 407.9 and total <= 474:
                return 'Class IV'
            elif total > 474 and total <= 540.2:
                return 'Class III'
            elif total > 540.2 and total <= 606.3:
                return 'Class II'
            elif total > 606.3 and total <= 672.5:
                return 'Class 1'
            elif total > 672.5 and total <= 744.1:
                return 'Master'
            elif total > 744.1 and total <= 810.2:
                return 'Elite' 
            elif  total > 810.2:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='67.5' and equip=='Raw' and ac=='Open':
            if total > 446.5 and total <= 523.6:
                return 'Class IV'
            elif total > 523.6 and total <= 595.3:
                return 'Class III'
            elif total > 595.3 and total <= 672.5:
                return 'Class II'
            elif total > 672.5 and total <= 744.1:
                return 'Class 1'
            elif total > 744.1 and total <= 815.8:
                return 'Master'
            elif total > 815.8 and total <= 892.9:
                return 'Elite' 
            elif total > 892.9:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='75' and equip=='Raw' and ac=='Open':
            if total > 479.6 and total <= 562.2:
                return 'Class IV'
            elif total > 562.2 and total <= 633.9:
                return 'Class III'
            elif total > 633.9 and total <= 727.6:
                return 'Class II'
            elif total > 727.6 and total <= 804.7:
                return 'Class 1'
            elif total > 804.7 and total <= 881.9:
                return 'Master'
            elif total > 881.9 and total <= 964.6:
                return 'Elite' 
            elif  total > 964.6:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='82.5' and equip=='Raw' and ac=='Open':
            if total > 518.1 and total <= 606.3:
                return 'Class IV'
            elif total > 606.3 and total <= 689:
                return 'Class III'
            elif total > 689 and total <= 777.2:
                return 'Class II'
            elif total > 777.2 and total <= 865.4:
                return 'Class 1'
            elif total > 865.4 and total <= 948:
                return 'Master'
            elif total > 948 and total <= 1030.7:
                return 'Elite' 
            elif total > 1030.7:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='90' and equip=='Raw' and ac=='Open':
            if total > 556.7 and total <= 633.9:
                return 'Class IV'
            elif total > 633.9 and total <= 738.6:
                return 'Class III'
            elif total > 738.6 and total <= 826.8:
                return 'Class II'
            elif total > 826.8 and total <= 920.5:
                return 'Class 1'
            elif total > 920.5 and total <= 1019.7:
                return 'Master'
            elif total > 1019.7 and total <= 1107.9:
                return 'Elite' 
            elif total > 1107.9:
                return 'Int Elite' 

        if event =='Full-Power' and wc =='100' and equip=='Raw' and ac=='Open':
            if total > 584.3 and total <= 678:
                return 'Class IV'
            elif total > 678 and total <= 777.2:
                return 'Class III'
            elif total > 777.2 and total <= 870.9:
                return 'Class II'
            elif total > 870.9 and total <= 970.1:
                return 'Class 1'
            elif total > 970.1 and total <= 1069.3:
                return 'Master'
            elif total > 1069.3 and total <= 1168.5:
                return 'Elite' 
            elif total > 1168.5:
                return 'Int Elite'

        if event =='Full-Power' and (wc =='110' or wc == '110+') and equip=='Raw' and ac=='Open':
            if total > 617.3 and total <= 716.6:
                return 'Class IV'
            elif total > 716.6 and total <= 821.3:
                return 'Class III'
            elif total > 821.3 and total <= 920.5:
                return 'Class II'
            elif total > 920.5 and total <= 1025.2:
                return 'Class 1'
            elif total > 1025.2 and total <= 1129.9:
                return 'Master'
            elif total > 1129.9 and total <= 1234.6:
                return 'Elite' 
            elif total > 1234.6:
                return 'Int Elite'

        if event =='Full-Power' and wc =='44' and equip=='Raw' and ac=='Open':
            if total > 314.2 and total <= 363.8:
                return 'Class IV'
            elif total > 363.8 and total <= 418.9:
                return 'Class III'
            elif total > 418.9 and total <= 468.5:
                return 'Class II'
            elif total > 468.5 and total <= 523.6:
                return 'Class 1'
            elif total > 523.6 and total <= 573.3:
                return 'Master'
            elif total > 573.3 and total <= 628.4:
                return 'Elite' 
            elif total > 628.4:
                return 'Int Elite'

        if event =='Full-Power' and wc =='48' and equip=='Raw' and ac=='Open':
            if total > 336.3 and total <= 391.4:
                return 'Class IV'
            elif total > 391.4 and total <= 446.5:
                return 'Class III'
            elif total > 446.5 and total <= 507.1:
                return 'Class II'
            elif total > 507.1 and total <= 562.2:
                return 'Class 1'
            elif total > 562.2 and total <= 617.3:
                return 'Master'
            elif total > 617.3 and total <= 672.5:
                return 'Elite' 
            elif total > 672.5:
                return 'Int Elite'

## Main function to perform data cleaning & transformation
## which involves applying the above functions, correcting data types, and replacing Nulls
def clean_data(x, fed):
    x[['Squat (KG)','Bench (KG)','Deadlift (KG)']] = x[['Squat (KG)','Bench (KG)','Deadlift (KG)']].replace({0:np.nan}).astype(float)
    x['Total (KG)'] = x['Total (KG)'].replace({'':0}).astype(float)
    x.Date = pd.to_datetime(x.Date)
    x['Number of Lifters'] = x['Number of Lifters'].astype(int)
    x= x[~x.Sex.isna()]
    if fed=='usapl':
        x['Age Division']= x.Division.apply(return_age_division_usapl)
    if fed =='uspa':
        x['Age Division']= x.Division.apply(return_age_division_uspa)
    x['Division Category'] = x.Division.apply(return_category)
    x.loc[x['Age'].str.contains('~'),'Age'] = x.loc[x['Age'].str.contains('~')].Age.apply(lambda x: x.strip('~'))
    x['Age'] = x['Age'].replace({'':0}).astype(int).replace({0:np.nan})
    x['Weight (KG)'] = x['Weight (KG)'].replace({'':0}).astype(float).replace({0:np.nan})
    x['Dots'] = x.Dots.replace({'':0}).astype(float).replace({0:np.nan})
    x['Dots'] = x.Dots.replace({'':0}).astype(float).replace({0:np.nan})
    for i in ['Squat (KG)','Bench (KG)','Deadlift (KG)','Total (KG)','Weight (KG)']:
        x[i.strip('(KG)') + str('(LB)')] = x[i] *  2.20462262
    x['Date Refreshed']= datetime.datetime.now().date()
    x['Classification'] = x.apply(lambda x: classification(x['Sex'],x['Division Category'], \
                                                                   x['Weight Class'], x['Equip'],x['Age Division'], x['Total (LB)']),axis=1)
    x['City'] = x[~x.City.isna()].City.apply(lambda x: x.split('\n')[0])
    return x
usapl=clean_data(df_usapl,'usapl')
uspa=clean_data(df_uspa,'uspa')

In [24]:
uspa.head()

,Division,Lifter Name,Place,Sex,Age,Equip,Weight Class,Weight (KG),Squat (KG),Bench (KG),Deadlift (KG),Total (KG),Dots,Country,State,Date,YM,City,Number of Lifters,Competition,Age Division,Division Category,Squat (LB),Bench (LB),Deadlift (LB),Total (LB),Weight (LB),Date Refreshed,Classification
0,Women Raw 75 Open,Liz Pagel,1,F,28.0,Raw,75,73.7,97.5,72.5,137.5,307.5,295.56,USA,CA,2010-12-11,2010/12,Camp Pendleton,113,Camp Pendleton Powerlifting Championship,Open,Full-Power,214.950705,159.835140,303.135610,677.921456,162.480687,2023-05-15,Class II
1,Women Raw 90+ Open,I-Esha Scott,1,F,27.0,Raw,90+,91.4,142.5,85.0,155.0,382.5,328.44,USA,CA,2010-12-11,2010/12,Camp Pendleton,113,Camp Pendleton Powerlifting Championship,Open,Full-Power,314.158723,187.392923,341.716506,843.268152,201.502507,2023-05-15,None
2,Men Raw 75 Juniors 13-15,Zack Smith,1,M,15.0,Raw,75,69.7,82.5,60.0,117.5,260.0,195.35,USA,CA,2010-12-11,2010/12,Camp Pendleton,113,Camp Pendleton Powerlifting Championship,Junior 13-15,Full-Power,181.881366,132.277357,259.043158,573.201881,153.662197,2023-05-15,None
3,Men Raw 75 Juniors 16-17,John Hansen Jr,DQ,M,17.0,Raw,75,74.8,142.5,NaN,172.5,0.0,NaN,USA,CA,2010-12-11,2010/12,Camp Pendleton,113,Camp Pendleton Powerlifting Championship,Junior 16-17,Full-Power,314.158723,NaN,380.297402,0.000000,164.905772,2023-05-15,None
4,Men Raw 125 Juniors 16-17,Tyler Sage,1,M,17.0,Raw,125,122.8,227.5,152.5,217.5,597.5,341.73,USA,CA,2010-12-11,2010/12,Camp Pendleton,113,Camp Pendleton Powerlifting Championship,Junior 16-17,Full-Power,501.551646,336.204950,479.505420,1317.262015,270.727658,2023-05-15,None


In [25]:
## Classify each dataframe based on their federation and concat them to make one dataframe
usapl['Federation'] = 'USAPL'
uspa['Federation'] = 'USPA'
final_df = pd.concat([usapl,uspa])
final_df = final_df.reset_index().drop('index',axis=1)
final_df.to_csv('both_federations.csv')

In [26]:
## Calculates number of new members per year
def new_member(x,fed,min_date,cur_year):
    new_members_m=[]
    new_members_f = []
    new_members_t = []
    names_list=set()
    if fed=='USPA':
        for i in np.arange(min_date, cur_year, 1): ## Loops through every year
            names = set(x[x.Date.apply(lambda x: x.year) == i]['Lifter Name'].unique()) ## Grabs all unique lifter names for that year
            difference = names - names_list ## Finds all lifters who have not lifted in the federation before
            subset = x[x['Lifter Name'].isin(difference)]
            names_list.update(names)
            new_members_m.append(subset[subset.Sex=='M']['Lifter Name'].nunique()) ## Counts # of new male lifters
            new_members_f.append(subset[subset.Sex=='F']['Lifter Name'].nunique()) ## Counts # of new female lifters
            new_members_t.append(len(difference)) ## Counts # of total new lifters
        m = pd.DataFrame([new_members_m, np.arange(2010,2023,1)]).T.rename({0:'Male New Members',1:'Year'},axis=1)
        f = pd.DataFrame([new_members_f, np.arange(2010,2023,1)]).T.rename({0:'Female New Members',1:'Year'},axis=1)
        t = pd.DataFrame([new_members_t, np.arange(2010,2023,1)]).T.rename({0:'Total New Members',1:'Year'},axis=1)
    elif fed=='USAPL':
        for i in np.arange(min_date, cur_year,1):
            names = set(x[x.Date.apply(lambda x: x.year) == i]['Lifter Name'].unique())
            difference = names - names_list
            subset = x[x['Lifter Name'].isin(difference)]
            names_list.update(names)
            new_members_m.append(subset[subset.Sex=='M']['Lifter Name'].nunique())
            new_members_f.append(subset[subset.Sex=='F']['Lifter Name'].nunique())
            new_members_t.append(len(difference))
        m = pd.DataFrame([new_members_m, np.arange(1997,2023,1)]).T.rename({0:'Male New Members',1:'Year'},axis=1)
        f = pd.DataFrame([new_members_f, np.arange(1997,2023,1)]).T.rename({0:'Female New Members',1:'Year'},axis=1)
        t = pd.DataFrame([new_members_t, np.arange(1997,2023,1)]).T.rename({0:'Total New Members',1:'Year'},axis=1)
    return m.merge(f,on='Year').merge(t,on='Year')
new_members_uspa = new_member(uspa, 'USPA', min_uspa, cur_year)
new_members_uspa['Federation']='USPA'
new_members_usapl = new_member(usapl,'USAPL', min_usapl, cur_year)
new_members_usapl['Federation'] = 'USAPL'
new_members = pd.concat([new_members_uspa,new_members_usapl]).reset_index().drop('index',axis=1)
new_members.to_csv('new_members.csv')

In [27]:
new_members.head()

,Male New Members,Year,Female New Members,Total New Members,Federation
0,122,2010,12,134,USPA
1,686,2011,147,833,USPA
2,1044,2012,206,1250,USPA
3,1606,2013,380,1986,USPA
4,2487,2014,847,3333,USPA


In [44]:
## Calculates number of new cities that host powerlifting competitions per year
def new_cities_(x, fed, min_date, cur_year):
    new_states = []
    new_cities = []
    cities_list=set()
    state_list = set()
    years_s = []
    years_c = []
    states_ = []
    if fed=='USPA':
        for i in np.arange(min_date, cur_year, 1): ## Loops through every year
            cities = x[x.Date.apply(lambda x: x.year) == i][['City','State']] 
            difference_c = set(cities.City.unique()) -cities_list ## Finds new cities that host a competition for that year and updates dict
            cities_list.update(set(cities.City.unique()))
            for city in cities_list: ## Loop through cities and append to new_cities_list & the year & state
                if city ==None:
                    continue
                new_cities.append(city)
                years_c.append(i)
                try:
                    states_.append(x[x.City==city]['State'].unique()[0])
                except:
                    pass
    elif fed =='USAPL':
        for i in np.arange(min_date, cur_year,1):
            cities = x[x.Date.apply(lambda x: x.year) == i][['City','State']]
            difference_c = set(cities.City.unique()) -cities_list
            cities_list.update(set(cities.City.unique()))
            for city in cities_list:
                if city ==None:
                    continue
                new_cities.append(city)
                years_c.append(i)
                try:
                    states_.append(x[x.City==city]['State'].unique()[0])
                except:
                    pass
    return pd.DataFrame([new_cities,years_c,states_]).T.rename({0:'City',1:'Date',2:'State'},axis=1)
new_cities_uspa = new_cities_(uspa,'USPA', min_uspa, cur_year)
new_cities_uspa['Federation'] = 'USPA'
new_cities_usapl = new_cities_(usapl,'USAPL', min_usapl, cur_year)
new_cities_usapl['Federation'] = 'USAPL'
new_cities=pd.concat([new_cities_uspa,new_cities_usapl]).reset_index().drop('index',axis=1)
new_cities.to_csv('new_cities.csv')

##### 

In [45]:
new_cities.head()

,City,Date,State,Federation
0,Saratoga,2010,CA,USPA
1,San Luis Obispo,2010,CA,USPA
2,Camp Pendleton,2010,CA,USPA
3,Branson,2011,MO,USPA
4,Port St. Lucie,2011,FL,USPA
